# Demo usage
Start by importing the ```CCalendar``` class from ```optimal_calendar```. Then, create an instance of this class. The constructor of ```CCalendar``` takes two _optional_ arguments, namely
- ```lecture_types```: contains all keywords that indicate a lecture. Default value: ```["hoorcollege"]``` 
- ```prac_sem_types```: contains all keywords that indicate a practical seminar group (or related event). Default value: ```["laptopcollege", "werkcollege", "presentatie"]```

Both arguments are used to _classify_ events while reading any data. Check your data to determine whether you need to pass your own lists as arguments.


In [1]:
from optimal_calendar import CCalendar

In [2]:
cal = CCalendar()

## Ignore certain data
The data to be loaded must be in ```xlsx``` format, specifically the format the UvA uses for it's course schedules. We can use the method ```CCalendar.load_course_from_excel```. This method takes in four arguments, which are 
- ```title```: title of the course
- ```path```: path to excel file
- ```ignore_type``` and ```ignore_description```

[Experimental | not yet tested in details]. When loading in the data, it is possible to ignore certain rows if they contain any of the specified keywords. We can ignore event types, e.g. PAV lectures, using ```ignore_type```. You might want to check your dataset to know which types to ignore (see the ```Type``` field in the excel file). In addition, we can ignore rows if their description contains any keyword in ```ignore_description```. Those lists can be different for every course. The default is an empty list, so no row will be ignored. 

Thus, the following lines are equivalent
```
cal.load_course_from_excel(path="", title="")
cal.load_course_from_excel(path="", title="", ignore_description=[])
```

This feature might come in handy if you are not interested in lectures (e.g. you are a TA and are only interested in the practical seminars of the associated course). 

In [3]:
BASE = "./schedules" # folder containing the excel files
EXTENS = "xlsx"

ignore = []
ignore_description = ["optional", "reistijd", "klankbordgroep", "pav"]

# examples
cal.load_course_from_excel(
    path=f"{BASE}/pss.{EXTENS}", 
    title="pss", 
    ignore_description=ignore_description, 
    ignore_type=["hoorcollege"] # e.g. for when you are a TA and want to see which groups fit your schedule
)
cal.load_course_from_excel(path=f"{BASE}/iki.{EXTENS}", title="iki", ignore_description=ignore_description)
cal.load_course_from_excel(path=f"{BASE}/mdb.{EXTENS}", title="mdb", ignore_description=ignore_description)
cal.load_course_from_excel(path=f"{BASE}/nwo.{EXTENS}", title="nwo", ignore_description=ignore_description)

print(cal)

Calendar contains pss, iki, mdb, nwo and 0 others events


# Finding combinations of practical seminar groups
Now, we can complete our goal: finding combinations of practical seminar groups, such that there is no overlap between them (lectures, exams etc. are being accounted for as well). We only need to use the method ```CCalendar.find_all_schedules```, which takes three optional arguments.
- ```format_output```: formats output. Default is True and is recommended.
- ```start_time```: output only contains seminar groups such that no seminar group starts before ```start_time```. No start time is set by default. Only uses 24h format, e.g. ```12:05```. 
- ```end_time```: output only contains seminar groups such that no seminar group ends after ```end_time```. No start time is set by default. Only uses 24h format, e.g. ```12:05```.
The arguments ```start_time``` and ```end_time``` must be used together, i.e. it will not work if you only specify one argument. You could, however, set the ```start_time``` at ```00:00``` if you do not care about starting time. Similarily, set ```24:00``` for ```end_time```.

The method returns a Pandas DataFrame object, simplifying the process of filtering (un)wanted combinations.

In [4]:
df = cal.find_all_schedules(
    start_time="10:00",
    end_time="17:00"
)

print(f"There are {len(df)} possible schedules")
df.head(10) # note that only the first 10 rows are shown below

There are 96 possible schedules


,iki,mdb,nwo,pss
0,E,H,D,A
1,E,H,D,B
2,E,H,D,C
3,E,H,D,D
4,E,H,D,E
5,E,H,D,F
6,E,H,D,G
7,E,H,D,H
8,E,H,D,I
9,E,H,D,J


In [5]:
# example: see all combinations such that iki group is E and pss group is A
df[(df["iki"] == "E") & (df["pss"] == "A")]

,iki,mdb,nwo,pss
0,E,H,D,A
12,E,I,D,A
